<a href="https://colab.research.google.com/github/npnkbabu/python/blob/master/Tobacco_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3482 images with 10 classes

 1. Memo - 0
 2. News - 1
 3. Note - 2
 4. Report - 3
 5. Resume - 4
 6. Scientific - 5
 7. Advertisement - 6
 8. Email - 7
 9. Form - 8
 10. Letter - 9


In [0]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [0]:
import keras.backend.tensorflow_backend as tfback
def _get_available_gpus():  

    if tfback._LOCAL_DEVICES is None:  
        devices = tf.config.list_logical_devices()  
        tfback._LOCAL_DEVICES = [x.name for x in devices]  
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]


tfback._get_available_gpus = _get_available_gpus

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
import tensorflow as tf

from keras.layers import Conv2D,Dense,MaxPooling2D,Flatten,Dropout,BatchNormalization, Activation
from keras.optimizers import SGD,RMSprop,Adam
from keras.models import Sequential
from keras.utils.np_utils import to_categorical

from hyperas.distributions import choice, uniform
from hyperas import optim
from hyperopt import Trials,STATUS_OK, tpe

import h5py

In [0]:
#LeNet-5 
#input-->Conv2D-->Maxpool-->Conv2d-->Maxpool-->Flatten-->Flatten->Dense-->Desnse(10)
#input-->Conv2D-->Conv2D-->maxpool-->Conv2D-->Conv2D-->Maxpool-->Flatten-->flatten-->Dense-->Dense(10)

In [0]:
#provide X_train,y_train,X_val,y_val
def data():
    # Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    link='https://drive.google.com/open?id=1MlmVyjNQxQRakD7HSFWUa6vJ3J36QBcY'
    fluff, id = link.split('=')
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('Tobacco3482_random.h5') 
    with(h5py.File(r'Tobacco3482_random.h5','r')) as hf:
        X = hf.get('Xdata')
        y = hf.get('ydata')
        X = np.array(X)
        y = np.array(y)
      
    #X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)
    X_train,X_val,y_train,y_val = train_test_split(X ,y ,test_size=0.2,random_state=2)

    #preprocessing

    X_train = X_train.astype('float32')
    #X_test = X_test.astype('float32')
    X_val = X_val.astype('float32')

    X_train /= 255
    #X_test /= 255
    X_val /= 255


    X_train = X_train.reshape(-1,X_train.shape[1],X_train.shape[2],1)
    #X_test = X_test.reshape(-1,X_test.shape[1],X_test.shape[2],1)
    X_val = X_val.reshape(-1,X_val.shape[1],X_val.shape[2],1)

    '''
    print(X_train.shape,'\r\n')
    print(y_train.shape,'\r\n')
    print(X_val.shape,'\r\n')
    print(y_val.shape,'\r\n')
    '''
    return X_train,y_train,X_val,y_val

In [23]:
#X_train,y_train,X_val,y_val = data()
X_train.shape

(2785, 299, 299, 1)

In [0]:
def model_arch1(X_train,y_train,X_val,y_val):
    model = Sequential([
    Conv2D(8,kernel_size=(3,3),padding='same'),
    MaxPooling2D(pool_size=(2,2),strides=2, padding='valid'),

    Conv2D(16,kernel_size=(5,5),padding='valid',activation='relu'),
    MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'),

    Flatten(),
    Dense(64,activation='relu'),
    Dense(10,activation='softmax')])
    #print (model.summary())

    model.compile(Adam(lr=0.01),'categorical_crossentropy',metrics=['accuracy'])
    model.fit(X_train,y_train,batch_size={{choice([16,32])}},epochs=3,verbose=2,validation_data=(X_val,y_val))
    score,acc = model.evaluate(X_val,y_val,verbose=0)
    print('test accuracy :',acc)
    return {'loss':-acc, 'status':STATUS_OK, 'model':model}


In [0]:
def model_Vgg(X_train,y_train,X_val,y_val):
    model = Sequential([
    Conv2D(16,kernel_size=(3,3),padding='same'),
    Conv2D(16,kernel_size=(3,3),padding='same'),
    MaxPooling2D(pool_size=(2,2),strides=2, padding='valid'),

    Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu'),
    Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu'),
    MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'),

    Flatten(),
    Dense(64,activation='relu'),
    Dense(10,activation='softmax')])
    #print (model.summary())

    model.compile(Adam(lr=0.01),'categorical_crossentropy',metrics=['accuracy'])
    model.fit(X_train,y_train,batch_size={{choice([16,32])}},epochs=3,verbose=2,validation_data=(X_val,y_val))
    score,acc = model.evaluate(X_val,y_val,verbose=0)
    print('test accuracy :',acc)
    return {'loss':-acc, 'status':STATUS_OK, 'model':model}

In [0]:
def model(X_train,y_train,X_val,y_val):
    model = Sequential()
    model_choice = {{choice(['one', 'two'])}}
    if model_choice == 'one':
        model.add(Conv2D(16, kernel_size=3, activation='relu',padding='same',   data_format='channels_first'))
        model.add(Conv2D(16, kernel_size=3, activation='relu',padding='same'))
        model.add(MaxPooling2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
        model.add(Conv2D(32, kernel_size=3, activation='relu'))
        model.add(Conv2D(32, kernel_size=3, activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
    elif model_choice == 'two':
        model.add(Conv2D(32, kernel_size=3, activation='relu',padding='same',  data_format='channels_first'))
        model.add(Conv2D(32, kernel_size=3, activation='relu',padding='same'))
        model.add(MaxPooling2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
        model.add(Conv2D(64, kernel_size=3, activation='relu'))
        model.add(Conv2D(64, kernel_size=3, activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
    
    model.add(Flatten())
    model.add(Dense({{choice([256, 512,1024])}}, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout({{uniform(0, 1)}}))
    choiceval = {{choice(['one', 'two'])}}
    if choiceval == 'two':
        model.add(Dense({{choice([256, 512,1024])}}, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout({{uniform(0, 1)}}))
    
    model.add(Dense(10, activation='softmax'))
    
    adam = Adam(lr=0.001)
    
    
    model.compile(Adam(lr=0.01),'categorical_crossentropy',metrics=['accuracy'])
    model.fit(X_train,y_train,batch_size={{choice([16,32])}},epochs=15,verbose=2,validation_data=(X_val,y_val))
    score,acc = model.evaluate(X_val,y_val,verbose=0)
    print('test accuracy :',acc)
    return {'loss':-acc, 'status':STATUS_OK, 'model':model}

In [0]:
def execution():
  # Authenticate and create the PyDrive client.
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  link='https://drive.google.com/open?id=1kmeGb48PyDjfXfxtV68YQUl1O_ENhWaM'
  fluff, id = link.split('=')
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('Tobacco_new.ipynb') 
  best_run, best_model = optim.minimize( model, data,algo=tpe.suggest,max_evals=3,trials=Trials(),notebook_name='Tobacco_new')
  return best_run,best_model

In [19]:
best_run, best_model = execution()

>>> Imports:
#coding=utf-8

try:
    from pydrive.auth import GoogleAuth
except:
    pass

try:
    from pydrive.drive import GoogleDrive
except:
    pass

try:
    from google.colab import auth
except:
    pass

try:
    from oauth2client.client import GoogleCredentials
except:
    pass

try:
    import keras.backend.tensorflow_backend as tfback
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout, BatchNormalization, Activation
except:
    pass

try:
    from keras.optimizers import SGD, RMSprop, Adam
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.utils.np_utils import to_categorical
except:
    pass

try:
  

In [20]:
best_run

{'Dense': 0,
 'Dense_1': 2,
 'Dropout': 0.44390441345306564,
 'Dropout_1': 0.039825089762879906,
 'Dropout_2': 0.11245974166556161,
 'Dropout_3': 0.3207527760045966,
 'Dropout_4': 0.7342146978592597,
 'Dropout_5': 0.692539034315719,
 'batch_size': 0,
 'model_choice': 1,
 'model_choice_1': 0}

In [28]:
cnn1 = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu',padding='same',input_shape=(299,299,1)),
    Conv2D(32, kernel_size=(3, 3), activation='relu',padding='same'),
    MaxPooling2D(pool_size=(2, 2),strides=2),
    Dropout(0.2),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2),strides=2),
    Dropout(0.3),
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.7),
    Dense(10, activation='softmax')
])
cnn1.compile(Adam(lr=0.01),'categorical_crossentropy',metrics=['accuracy'])
cnn1.fit(X_train,y_train,batch_size=32,epochs=15,verbose=2,validation_data=(X_val,y_val))
#score,acc = cnn1.evaluate(X_val,y_val,verbose=0)
#print('test accuracy :',acc)
#return {'loss':-acc, 'status':STATUS_OK, 'model':model}

Train on 2785 samples, validate on 697 samples
Epoch 1/15
 - 32s - loss: 2.1517 - accuracy: 0.3522 - val_loss: 45.1936 - val_accuracy: 0.1306
Epoch 2/15
 - 27s - loss: 1.5545 - accuracy: 0.4995 - val_loss: 47.3055 - val_accuracy: 0.1263
Epoch 3/15
 - 27s - loss: 1.4220 - accuracy: 0.5562 - val_loss: 67.6318 - val_accuracy: 0.0703
Epoch 4/15
 - 27s - loss: 1.2949 - accuracy: 0.5759 - val_loss: 45.6409 - val_accuracy: 0.0617
Epoch 5/15
 - 27s - loss: 1.1195 - accuracy: 0.6302 - val_loss: 8.7840 - val_accuracy: 0.2453
Epoch 6/15
 - 27s - loss: 1.0638 - accuracy: 0.6560 - val_loss: 4.5927 - val_accuracy: 0.2166
Epoch 7/15
 - 27s - loss: 1.0559 - accuracy: 0.6521 - val_loss: 1.9833 - val_accuracy: 0.5136
Epoch 8/15
 - 27s - loss: 0.8831 - accuracy: 0.7084 - val_loss: 4.0835 - val_accuracy: 0.2941
Epoch 9/15
 - 27s - loss: 0.6923 - accuracy: 0.7691 - val_loss: 5.0733 - val_accuracy: 0.2468
Epoch 10/15
 - 27s - loss: 0.5772 - accuracy: 0.8029 - val_loss: 10.4422 - val_accuracy: 0.1750
Epoch 1

In [29]:
score,acc = cnn1.evaluate(X_val,y_val,verbose=0)
print('test accuracy :',acc)

test accuracy : 0.5997130274772644
